# Import relevant libraries

In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import pickle
from model_architecture import get_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from sklearn.svm import SVC
from keras.models import Model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import cv2
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Preprocessing

#### take latest model

In [2]:
import joblib
model = get_model()
model.load_weights('./weights/final.weights.h5')

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


#### Take data

In [3]:
save_path = './'

with open(os.path.join(save_path, "train_fifth.pickle"), 'rb') as f:
    (train_data, train_labels) = pickle.load(f)

with open(os.path.join(save_path, "test_fifth.pickle"), 'rb') as f:
    (test_data, test_labels) = pickle.load(f)

#### Normalize

In [4]:
def normalize_image_numpy(image):
    # Convert the pixel values to floats between 0 and 1
    image = image.astype(np.float32) / 255.0
    return image

train_data_normalized = normalize_image_numpy(train_data)
test_data_normalized = normalize_image_numpy(test_data)

#### CNN feature extraction

In [5]:
# Function to extract features from the CNN model
def extract_features(model, X):
    feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
    features = feature_extractor.predict(X)
    return features

X_train_features = extract_features(model, train_data_normalized)
X_test_features = extract_features(model, test_data_normalized)

100/100 ━━━━━━━━━━━━━━━━━━━━ 497s 5s/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step


# Methods

### 1. CNN

In [6]:
# y_pred_1_f = model.predict(test_data_normalized)

# y_pred_1 = np.argmax(y_pred_1_f, axis=1)

In [7]:
# classification_rep_1 = classification_report(test_labels, y_pred_1)
# print("Classification Report:")
# print(classification_rep_1)

In [8]:
# from tabulate import tabulate

# def print_colorful_classification_report(classification_report):
#     report_lines = classification_report.split('\n')
#     headers = report_lines[0].split()
#     data = []

#     for line in report_lines[2:-5]:  # Exclude header and footer lines
#         row_data = line.split()
#         class_name = row_data[0]
#         row_data = [float(val) if val != 'accuracy' else val for val in row_data[1:]]
#         data.append([class_name] + row_data)

#     print(tabulate(data, headers=headers, tablefmt='fancy_grid', numalign='center'))


# print_colorful_classification_report(classification_rep_1)


### 2. KNN

In [9]:
def train_knn(features, y):
    knn_clf = KNeighborsClassifier(n_neighbors=4)  # You can adjust the number of neighbors as needed
    knn_clf.fit(features, y)
    return knn_clf

knn_classifier = train_knn(X_train_features, train_labels)

y_pred_6 = knn_classifier.predict(X_test_features)

accuracy_6 = accuracy_score(test_labels, y_pred_6)

print("Accuracy:", accuracy_6)

Accuracy: 0.838


In [21]:
joblib.dump(knn_classifier, 'knn_model.pkl')

# Load the KNN model from the file
loaded_knn_model = joblib.load('knn_model.pkl')

# Use the loaded model for prediction
y_pred_knn = loaded_knn_model.predict(X_test_features)

# Calculate accuracy
accuracy_knn = accuracy_score(test_labels, y_pred_knn)
print("KNN Accuracy:", accuracy_knn)

KNN Accuracy: 0.838


In [10]:
classification_rep_6 = classification_report(test_labels, y_pred_6)
print("Classification Report:")
print(classification_rep_6)

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       250
           1       0.89      0.89      0.89       250
           2       0.76      0.74      0.75       250
           3       0.88      0.84      0.86       250

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000



### 3. SVM

#### SVM{'gamma'='auto', 'kernel': 'rbf'}

In [11]:
def train_svm(features, y):
    svm_clf = SVC(kernel='rbf', gamma='auto')
    svm_clf.fit(features, y)
    return svm_clf

svm_classifier_1 = train_svm(X_train_features, train_labels)

y_pred_1 = svm_classifier_1.predict(X_test_features)

accuracy_1 = accuracy_score(test_labels, y_pred_1)
print("Accuracy:", accuracy_1)

Accuracy: 0.867


In [12]:
classification_rep_1 = classification_report(test_labels, y_pred_1)
print("Classification Report:")
print(classification_rep_1)

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       250
           1       0.93      0.90      0.91       250
           2       0.82      0.79      0.80       250
           3       0.86      0.90      0.88       250

    accuracy                           0.87      1000
   macro avg       0.87      0.87      0.87      1000
weighted avg       0.87      0.87      0.87      1000



#### SVM{'C': 0.1, 'kernel': 'linear'}

In [13]:
def train_svm(features, y):
    svm_clf = SVC(C = 0.1, kernel = 'linear')
    svm_clf.fit(features, y)
    return svm_clf

svm_classifier_2 = train_svm(X_train_features, train_labels)

y_pred_2 = svm_classifier_2.predict(X_test_features)

accuracy_2 = accuracy_score(test_labels, y_pred_2)
print("Accuracy:", accuracy_2)

Accuracy: 0.873


In [14]:
classification_rep_2 = classification_report(test_labels, y_pred_2)
print("Classification Report:")
print(classification_rep_2)

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.89      0.87       250
           1       0.93      0.90      0.91       250
           2       0.84      0.80      0.82       250
           3       0.88      0.91      0.89       250

    accuracy                           0.87      1000
   macro avg       0.87      0.87      0.87      1000
weighted avg       0.87      0.87      0.87      1000



#### SVM{'C': 100, 'degree': 2, 'kernel': 'poly'}

In [15]:
def train_svm(features, y):
    svm_clf = SVC(C = 100, kernel = 'poly', degree=2)
    svm_clf.fit(features, y)
    return svm_clf

svm_classifier_3 = train_svm(X_train_features, train_labels)

y_pred_3 = svm_classifier_3.predict(X_test_features)

accuracy_3 = accuracy_score(test_labels, y_pred_3)
print("Accuracy:", accuracy_3)

Accuracy: 0.878


In [22]:
import joblib
joblib.dump(svm_classifier_1, 'svm_model.pkl')

# Load the SVM model from the file
loaded_svm_model = joblib.load('svm_model.pkl')

# Use the loaded model for prediction
y_pred_3 = loaded_svm_model.predict(X_test_features)

# Calculate accuracy
accuracy_3 = accuracy_score(test_labels, y_pred_3)
print("Accuracy:", accuracy_3)

Accuracy: 0.867


In [16]:
classification_rep_3 = classification_report(test_labels, y_pred_3)
print("Classification Report:")
print(classification_rep_3)

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       250
           1       0.94      0.90      0.92       250
           2       0.84      0.81      0.82       250
           3       0.89      0.92      0.90       250

    accuracy                           0.88      1000
   macro avg       0.88      0.88      0.88      1000
weighted avg       0.88      0.88      0.88      1000



### 4. Adaboost {base classifier= random forest, n_estimators = 50}

In [17]:
def train_adaboost(features, y, n_estimators=10):
    base_classifier = RandomForestClassifier(n_estimators=100)
    ada_boost_clf = AdaBoostClassifier(estimator=base_classifier, n_estimators=n_estimators)
    ada_boost_clf.fit(features, y)
    return ada_boost_clf


ada_boost_classifier = train_adaboost(X_train_features, train_labels)

y_pred_4 = ada_boost_classifier.predict(X_test_features)

accuracy_4 = accuracy_score(test_labels, y_pred_4)
print("Accuracy:", accuracy_4)

Accuracy: 0.863


In [23]:
joblib.dump(ada_boost_classifier, 'adaboost_model.pkl')

# Load the AdaBoost model from the file
loaded_adaboost_model = joblib.load('adaboost_model.pkl')

# Use the loaded model for prediction
y_pred_adaboost = loaded_adaboost_model.predict(X_test_features)

# Calculate accuracy
accuracy_adaboost = accuracy_score(test_labels, y_pred_adaboost)
print("AdaBoost Accuracy:", accuracy_adaboost)

AdaBoost Accuracy: 0.863


In [18]:
classification_rep_4 = classification_report(test_labels, y_pred_4)
print("Classification Report:")
print(classification_rep_4)

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.87      0.87       250
           1       0.91      0.90      0.91       250
           2       0.81      0.80      0.80       250
           3       0.87      0.88      0.87       250

    accuracy                           0.86      1000
   macro avg       0.86      0.86      0.86      1000
weighted avg       0.86      0.86      0.86      1000



### 5. XGBOOST

In [19]:
# Define XGBoost parameters
params = {
    'objective': 'multi:softmax',  # Multi-class classification
    'num_class': 4,  # Number of classes
    'max_depth': 3,  # Maximum depth of each tree
    'eta': 0.1,  # Learning rate
    'subsample': 0.5,  # Subsample ratio of the training instance
    'colsample_bytree': 0.5,  # Subsample ratio of columns when constructing each tree
    'eval_metric': 'merror'  # Evaluation metric (multiclass error rate)
}

# Convert the dataset into DMatrix format, which is optimized for XGBoost
dtrain = xgb.DMatrix(X_train_features, label=train_labels)
dtest = xgb.DMatrix(X_test_features)

# Train the XGBoost model
num_rounds = 500  # Number of boosting rounds
xgb_model = xgb.train(params, dtrain, num_rounds)

# Make predictions
y_pred_5 = xgb_model.predict(dtest)

# Calculate accuracy
accuracy_5 = accuracy_score(test_labels, y_pred_5)
print("Accuracy:", accuracy_5)

Accuracy: 0.867


In [24]:
# Save the trained XGBoost model to a file
joblib.dump(xgb_model, 'xgboost_model.pkl')

# Load the XGBoost model from the file
loaded_xgboost_model = joblib.load('xgboost_model.pkl')

# Use the loaded model for prediction
dtest = xgb.DMatrix(X_test_features)
y_pred_xgboost = loaded_xgboost_model.predict(dtest)

# Calculate accuracy
accuracy_xgboost = accuracy_score(test_labels, y_pred_xgboost)
print("XGBoost Accuracy:", accuracy_xgboost)

XGBoost Accuracy: 0.867


In [20]:
classification_rep_5 = classification_report(test_labels, y_pred_5)
print("Classification Report:")
print(classification_rep_5)

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.89      0.87       250
           1       0.92      0.89      0.91       250
           2       0.82      0.80      0.81       250
           3       0.89      0.89      0.89       250

    accuracy                           0.87      1000
   macro avg       0.87      0.87      0.87      1000
weighted avg       0.87      0.87      0.87      1000

